Base

In [ ]:
# Filename: Comparison_Euler_vs_Milstein_vs_QE.ipynb

# Import necessary libraries and modules
import numpy as np
import matplotlib.pyplot as plt
from EulerScheme import EulerScheme
from MilsteinScheme import MilsteinScheme
from QEScheme import QEScheme  # Ensure QEScheme is defined
from HestonModel import HestonModel
from MonteCarloSimulator import MonteCarloSimulator
from MethodComparison import MethodComparison

from scipy.stats import norm
from scipy.integrate import quad

# Define functions for semi-analytical Heston model pricing

def heston_characteristic_function(u, T, S0, v0, r, kappa, theta, sigma, rho):
    """
    Calculate the characteristic function phi(u) for the Heston model.
    """
    i = complex(0, 1)
    
    # Calculate d and g as per the Heston characteristic function definition
    d = np.sqrt((rho * sigma * i * u - kappa)**2 + (u**2 + i * u) * sigma**2)
    g = (kappa - rho * sigma * i * u - d) / (kappa - rho * sigma * i * u + d)
    
    # Calculate C and D, two core components of the characteristic function
    C = r * i * u * T + (kappa * theta / sigma**2) * ((kappa - rho * sigma * i * u - d) * T - 
                                                      2 * np.log((1 - g * np.exp(-d * T)) / (1 - g)))
    D = ((kappa - rho * sigma * i * u - d) / sigma**2) * ((1 - np.exp(-d * T)) / (1 - g * np.exp(-d * T)))
    
    # Return the value of the characteristic function as a complex exponential
    return np.exp(C + D * v0 + i * u * np.log(S0))

def heston_integrand(u, S0, K, T, r, v0, kappa, theta, sigma, rho, phi_num):
    """
    Calculate the integrand in the Heston model for European call option pricing.
    Parameter phi_num decides if we calculate phi1 or phi2.
    """
    i = complex(0, 1)
    char_func_val = heston_characteristic_function(u - (i if phi_num == 1 else 0), T, S0, v0, r, kappa, theta, sigma, rho)
    numerator = np.exp(-i * u * np.log(K)) * char_func_val
    denominator = i * u
    return (numerator / denominator).real

def heston_call_price(S0, K, T, r, v0, kappa, theta, sigma, rho):
    """
    Calculate the European call option price using the Heston semi-analytical formula.
    """
    # Use two characteristic functions phi1 and phi2 to calculate the option price
    integral_phi1, _ = quad(lambda u: heston_integrand(u, S0, K, T, r, v0, kappa, theta, sigma, rho, 1), 0, np.inf)
    integral_phi2, _ = quad(lambda u: heston_integrand(u, S0, K, T, r, v0, kappa, theta, sigma, rho, 2), 0, np.inf)
    
    # Calculate the call option price using both integrals
    call_price = S0 * (0.5 + integral_phi1 / np.pi) - K * np.exp(-r * T) * (0.5 + integral_phi2 / np.pi)
    return call_price

# Initialize parameters for the Heston model
S0 = 100        # Initial asset price
v0 = 0.04       # Initial volatility
kappa = 2.0     # Mean reversion rate
theta = 0.04    # Long-term mean
sigma = 0.3     # Volatility of volatility
rho = -0.7      # Correlation
r = 0.05        # Risk-free interest rate
T = 1           # Time to maturity
K = 100         # Strike price

# Calculate the theoretical option price (true value) using the Heston model
true_price = heston_call_price(S0, K, T, r, v0, kappa, theta, sigma, rho)
print("Theoretical Call Option Price (Heston Model):", true_price)

true_price = 10  # Assuming the estimated true price for comparison purposes

# Define simulation parameters
num_paths = 100     # Number of simulation paths
num_steps = 1000     # Number of time steps

# Create an instance of the Heston model
heston_model = HestonModel(S0, v0, kappa, theta, sigma, rho, r)

# Create instances for Euler, Milstein, and QE discretization schemes
euler_scheme = EulerScheme(S0, v0, kappa, theta, sigma, rho, r)
milstein_scheme = MilsteinScheme(S0, v0, kappa, theta, sigma, rho, r)
qe_scheme = QEScheme(S0, v0, kappa, theta, sigma, rho, r)

# Use the MethodComparison class to compare the accuracy of the three schemes
comparison = MethodComparison(heston_model, T, K, true_price)

# Calculate the accuracy (error from true price) for each scheme
euler_accuracy = comparison.accuracy(euler_scheme, num_paths, num_steps)
milstein_accuracy = comparison.accuracy(milstein_scheme, num_paths, num_steps)
qe_accuracy = comparison.accuracy(qe_scheme, num_paths, num_steps)

print(f"Euler Scheme Accuracy (Error): {euler_accuracy}")
print(f"Milstein Scheme Accuracy (Error): {milstein_accuracy}")
print(f"QE Scheme Accuracy (Error): {qe_accuracy}")

"""
# Compare the convergence of the three schemes
step_sizes = [50, 100, 200, 1000]  # Different time step counts
euler_convergence = comparison.convergence(euler_scheme, num_paths, step_sizes)
milstein_convergence = comparison.convergence(milstein_scheme, num_paths, step_sizes)
qe_convergence = comparison.convergence(qe_scheme, num_paths, step_sizes)

# Plot convergence comparison
plt.figure(figsize=(10, 6))
plt.plot(step_sizes, euler_convergence, label='Euler Scheme Convergence', marker='o')
plt.plot(step_sizes, milstein_convergence, label='Milstein Scheme Convergence', marker='s')
plt.plot(step_sizes, qe_convergence, label='QE Scheme Convergence', marker='^')
plt.xlabel('Number of Steps')
plt.ylabel('Error')
plt.title('Convergence Comparison: Euler vs Milstein vs QE Scheme')
plt.legend()
plt.grid(True)
plt.show()

# Compare the stability of the three schemes
num_trials = 10
euler_stability = comparison.stability(euler_scheme, num_paths, num_steps, num_trials)
milstein_stability = comparison.stability(milstein_scheme, num_paths, num_steps, num_trials)
qe_stability = comparison.stability(qe_scheme, num_paths, num_steps, num_trials)

print(f"Euler Scheme Stability (Standard Deviation): {euler_stability}")
print(f"Milstein Scheme Stability (Standard Deviation): {milstein_stability}")
print(f"QE Scheme Stability (Standard Deviation): {qe_stability}")
"""


Theoretical Call Option Price (Heston Model): 2008.2521168395315
Euler Scheme Accuracy (Error): 0.04358301453838642
Milstein Scheme Accuracy (Error): 0.895583753333753
QE Scheme Accuracy (Error): 112.78046973138069


'\n# 比较三种方案的收敛性\nstep_sizes = [50, 100, 200, 1000]  # 不同的时间步数\neuler_convergence = comparison.convergence(euler_scheme, num_paths, step_sizes)\nmilstein_convergence = comparison.convergence(milstein_scheme, num_paths, step_sizes)\nqe_convergence = comparison.convergence(qe_scheme, num_paths, step_sizes)\n\n# 绘制收敛性比较图\nplt.figure(figsize=(10, 6))\nplt.plot(step_sizes, euler_convergence, label=\'Euler Scheme Convergence\', marker=\'o\')\nplt.plot(step_sizes, milstein_convergence, label=\'Milstein Scheme Convergence\', marker=\'s\')\nplt.plot(step_sizes, qe_convergence, label=\'QE Scheme Convergence\', marker=\'^\')\nplt.xlabel(\'Number of Steps\')\nplt.ylabel(\'Error\')\nplt.title(\'Convergence Comparison: Euler vs Milstein vs QE Scheme\')\nplt.legend()\nplt.grid(True)\nplt.show()\n\n# 比较三种方案的稳定性\nnum_trials = 10\neuler_stability = comparison.stability(euler_scheme, num_paths, num_steps, num_trials)\nmilstein_stability = comparison.stability(milstein_scheme, num_paths, num_steps, num_tri